<a href="https://colab.research.google.com/github/VipanchiKatthula/PySpark_OperationsOnAmazonTweets/blob/master/PySpark_Operations_on_Amazon_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null   #Install Spark

In [0]:
!wget -q http://apache.mirrors.pair.com/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz  #Get Spark Insatller

In [0]:
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz  #Untar Spark installer

***Install findspark - a python library to find Spark***

In [0]:
!pip install -q findspark   

***Setting environmental Variables***

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

***Create a local Spark session***

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

***Importing data and Selecting required columns***

In [0]:
data = spark.read.format("csv").option("header","true").load("drive/My\ Drive/Colab\ Notebooks/Data/Amazon_Responded_Oct05.csv")
data1 = data.select("id_str",'tweet_created_at','user_verified','favorite_count','retweet_count','text_')

***Number of rows after filtering for only verified users***

In [12]:
dat_filtered = data1.filter(data1.user_verified == 'True')
dat_filtered.count() #Selecting only those tweets with vaild/verified users

171797

In [14]:
dat_filtered.select("tweet_created_at").show(5,False)

+------------------------------+
|tweet_created_at              |
+------------------------------+
|Tue Nov 01 02:39:55 +0000 2016|
|Tue Nov 01 17:19:57 +0000 2016|
|Tue Nov 01 17:25:26 +0000 2016|
|Tue Nov 01 18:02:03 +0000 2016|
|Tue Nov 01 03:59:02 +0000 2016|
+------------------------------+
only showing top 5 rows



In [76]:
from pyspark.sql.functions import split   #Splitting the date column to parse month date and year.
a = split(dat_filtered["tweet_created_at"], ' ')
dat_filtered1 = dat_filtered.withColumn('Month', a.getItem(1))
dat_filtered1 = dat_filtered1.withColumn('Date', a.getItem(2))
dat_filtered1 = dat_filtered1.withColumn('Year', a.getItem(5))
dat_filtered1.show(5)

+--------------------+--------------------+-------------+--------------+-------------+--------------------+-----+----+----+
|              id_str|    tweet_created_at|user_verified|favorite_count|retweet_count|               text_|Month|Date|Year|
+--------------------+--------------------+-------------+--------------+-------------+--------------------+-----+----+----+
|'793281386912354304'|Tue Nov 01 02:39:...|         True|             0|            0|@SeanEPanjab I'm ...|  Nov|  01|2016|
|'793502854459879424'|Tue Nov 01 17:19:...|         True|             0|            0|@SeanEPanjab Plea...|  Nov|  01|2016|
|'793504235400884224'|Tue Nov 01 17:25:...|         True|             0|            0|@SeanEPanjab With...|  Nov|  01|2016|
|'793513446633533440'|Tue Nov 01 18:02:...|         True|             0|            0|@SeanEPanjab I'm ...|  Nov|  01|2016|
|'793301295255945216'|Tue Nov 01 03:59:...|         True|             0|            0|@aakashwangnoo Hi...|  Nov|  01|2016|
+-------

In [78]:
import pyspark.sql.functions as sq 
df = dat_filtered1.withColumn("tweet_created_at",sq.concat(sq.col("Month"), sq.lit(" "), sq.col("Date"),sq.lit(" "), sq.col("Year")))
df = df.select('id_str','tweet_created_at','user_verified',df.favorite_count.cast('int'), df.retweet_count.cast('int'),'text_')
df.show(5)

+--------------------+----------------+-------------+--------------+-------------+--------------------+
|              id_str|tweet_created_at|user_verified|favorite_count|retweet_count|               text_|
+--------------------+----------------+-------------+--------------+-------------+--------------------+
|'793281386912354304'|     Nov 01 2016|         True|             0|            0|@SeanEPanjab I'm ...|
|'793502854459879424'|     Nov 01 2016|         True|             0|            0|@SeanEPanjab Plea...|
|'793504235400884224'|     Nov 01 2016|         True|             0|            0|@SeanEPanjab With...|
|'793513446633533440'|     Nov 01 2016|         True|             0|            0|@SeanEPanjab I'm ...|
|'793301295255945216'|     Nov 01 2016|         True|             0|            0|@aakashwangnoo Hi...|
+--------------------+----------------+-------------+--------------+-------------+--------------------+
only showing top 5 rows



In [79]:
counts = df.groupby(df.tweet_created_at).agg(sq.count('id_str').alias("count_of_tweets"))
counts.show(5)

+----------------+---------------+
|tweet_created_at|count_of_tweets|
+----------------+---------------+
|     Dec 01 2016|            875|
|     Dec 25 2016|            433|
|     Feb 08 2017|            926|
|     Feb 19 2017|            725|
|     Mar 30 2017|           1109|
+----------------+---------------+
only showing top 5 rows



In [80]:
import pyspark
conf = pyspark.SparkConf()
sc = pyspark.SparkContext.getOrCreate(conf=conf)
from pyspark.sql import SQLContext
sqlcontext = SQLContext(sc)

counts.registerTempTable("tmpcounts")
counts_ordered = sqlcontext.sql("SELECT * FROM tmpcounts order by count_of_tweets desc limit 5")
counts_ordered.show()

+----------------+---------------+
|tweet_created_at|count_of_tweets|
+----------------+---------------+
|     Jan 03 2017|           1536|
|     Jan 10 2017|           1508|
|     Jan 11 2017|           1496|
|     Jan 12 2017|           1410|
|     Jan 06 2017|           1364|
+----------------+---------------+



In [81]:
df.registerTempTable("tmpdf")
sum_retweets = sqlcontext.sql("SELECT text_,favorite_count+ retweet_count as total from tmpdf where tweet_created_at = 'Jan 03 2017' order by total desc limit 100")
sum_retweets.show(10,False)
text = sum_retweets.toPandas()
a =text["text_"].tolist()

+--------------------------------------------------------------------------------------------------------------------------------------------+-----+
|text_                                                                                                                                       |total|
+--------------------------------------------------------------------------------------------------------------------------------------------+-----+
|@amazon worst shopping  experience,  no service, no substantial reply to complaints, no delivery for 1 week post guarantee date.            |5    |
|@ItsJosshA We always aim to deliver by the date given in your confirmation email. Have we missed that date? Any update in tracking?  ^NF    |3    |
|@ItsJosshA Oh no! I'm sorry! Please reach out to us so that we can look into options: https://t.co/hApLpMlfHN. ^JO                          |2    |
|@KStefl Sounds like you know what to add to your Halloween playlist for this year! ^BV                   

## ***Cleaning Function to remove twitter usernames and punctuation***

In [69]:
import nltk
import pandas as pd
import string
import re
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
from collections import Counter

def cleaning(textfile):
    y=textfile
    cleaned=[]
    for i in y:
      # print(i)
      i.replace("'", " ")  #step 1: replacing apostraphe
      i=re.sub(r"\@\w+"," ",i)   #step2: removing words starting with @
      cleaned.append(i.translate(str.maketrans(string.punctuation,' '*len(string.punctuation)))) # step 3: remove_punctuation                   
    output=[]
    for i in cleaned:
        output.append(" ".join([w.lower() for w in i.split()  if w.isalpha()]))    

    return output

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


***Word frequency and printing it to a .csv file***

In [75]:
cleaned_text = cleaning(a)
from pyspark import SparkContext
sc =SparkContext.getOrCreate()
k = cleaned_text
tuple(k)
rdd = sc.parallelize(k)
word_frequency_rdd = rdd.flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()
print(word_frequency_rdd)
tweet_word_frequency_rdd=pd.DataFrame(word_frequency_rdd)
tweet_word_frequency_rdd.to_csv(r"/content/tweet_word_frequency_rdd.csv")

[('worst', 1), ('no', 5), ('service', 1), ('substantial', 1), ('delivery', 6), ('week', 1), ('we', 46), ('always', 3), ('in', 13), ('confirmation', 1), ('have', 18), ('update', 2), ('tracking', 5), ('fancy', 1), ('already', 1), ('don', 5), ('look', 7), ('pass', 2), ('details', 6), ('https', 44), ('co', 45), ('lil', 1), ('sounds', 1), ('like', 7), ('know', 13), ('playlist', 1), ('this', 27), ('year', 2), ('bv', 2), ('d', 7), ('help', 9), ('when', 5), ('connect', 1), ('us', 28), ('matt', 1), ('friendshipgoals', 1), ('oh', 1), ('i', 23), ('out', 6), ('into', 6), ('options', 3), ('hear', 4), ('as', 4), ('let', 7), ('doesn', 5), ('arrive', 6), ('tomorrow', 2), ('an', 8), ('ar', 3), ('apologies', 2), ('incorrect', 1), ('item', 5), ('using', 2), ('above', 2), ('do', 4), ('further', 5), ('concerns', 1), ('ca', 1), ('thanks', 11), ('shout', 1), ('looking', 2), ('improve', 1), ('share', 1), ('specific', 1), ('feedback', 7), ('via', 4), ('form', 1), ('sj', 6), ('delay', 1), ('indicate', 1), ('new

## ***Task -2***

In [23]:
finddata = spark.read.format("csv").option("header","true").load("drive/My\ Drive/Colab\ Notebooks/Data/find_text.csv")
finddata.show(5,False)

+--------------------+----+
|id_str              |text|
+--------------------+----+
|'793270689780203520'|null|
|'793281386912354304'|null|
|'793299404975247360'|null|
|'793301295255945216'|null|
|'793315815411978240'|null|
+--------------------+----+
only showing top 5 rows



In [85]:
finddata.registerTempTable("ids")
data.registerTempTable("tweets")
out = sqlcontext.sql("SELECT DISTINCT I.id_str,T.text_ from ids I JOIN tweets T on I.id_str = T.id_str")
out.show(5,False)
output = out.toPandas()
output.to_csv(r"/content/join_output.csv")

+--------------------+-----------------------------------------------------------------------------------------------------------------------------------------+
|id_str              |text_                                                                                                                                    |
+--------------------+-----------------------------------------------------------------------------------------------------------------------------------------+
|'793382930085253121'|@mybharatraj Hi! Sorry about that. I'd like our team to look into this, please reach out to us via: https://t.co/vlvfJr4nN9 ^SG          |
|'793441656984903680'|@AmazonHelp done, please contact them and let them know I am waiting for my package.                                                     |
|'793517259880861696'|Your customer service sucks, @AmazonHelp. If you guys can't do your job, perhaps go back to school and learn something proper.           |
|'793533066157387776'|@flamablebro